In [3]:
import dotenv
import os
from langchain_openai import OpenAI

dotenv.load_dotenv()

True

In [4]:
SECRET_KEY = os.getenv("OPENAI_API_KEY")

llm = OpenAI(api_key=SECRET_KEY, temperature=.7) 

# First test

In [5]:
explanation = llm.invoke("Using the latest information, provide a buy or sell recommendation for the stock of Apple Inc. (AAPL). Add reasoning for your recommendation.")
explanation

"\n\nBased on the current information, I would recommend a buy for the stock of Apple Inc. (AAPL).\n\nFirstly, Apple recently announced its financial results for the first quarter of fiscal year 2021, and the numbers were impressive. The company posted a record-breaking revenue of $111.4 billion, a 21% increase from the same quarter last year. This was driven by strong demand for its iPhone, Mac, and iPad products. In addition, Apple's services segment, which includes the App Store and Apple Music, also saw a 24% growth in revenue.\n\nSecondly, Apple has a strong product pipeline for 2021, which includes the highly anticipated launch of the iPhone 12, as well as potential new products such as AirTags and AirPods Pro 2. These new products are expected to drive further revenue growth for the company.\n\nMoreover, Apple's financial position remains strong, with a cash balance of over $195 billion and a low debt-to-equity ratio of 1.36. This provides the company with the flexibility to inv

# Adding a prompt template

In [6]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(

    input_variables=["stock"],
    input_types={
        "stock": str,
    },
    template="Using the latest information, provide a buy or sell recommendation for the stock of {stock}. Add reasoning for your recommendation.",
)

prompt.format(stock="Amazon Inc. (AMZN)")

'Using the latest information, provide a buy or sell recommendation for the stock of Amazon Inc. (AMZN). Add reasoning for your recommendation.'

# Making this as a chain

In [7]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

chain.invoke(input={"stock": "Amazon Inc. (AMZN)"})

{'stock': 'Amazon Inc. (AMZN)',
 'text': "\n\nBased on the latest information, I would recommend a buy for the stock of Amazon Inc. (AMZN). Amazon's stock has been performing well recently, with its share price reaching an all-time high of $3,400 in September 2020. This is attributed to the company's strong financial performance, with its third-quarter earnings report showing a 37% increase in net sales compared to the same period last year.\n\nAdditionally, Amazon has been able to capitalize on the shift towards e-commerce and online shopping due to the COVID-19 pandemic. The company's Prime membership program has also seen significant growth, with over 150 million subscribers worldwide. This demonstrates the strong customer loyalty and potential for recurring revenue for Amazon.\n\nFurthermore, Amazon has been expanding its business beyond e-commerce into other sectors such as cloud computing, advertising, and media streaming. These efforts have been successful, with the company's cl

# Let's try to add a search function


In [8]:
from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
tool = Tool(
    name="Google Search",
    description="Search Google for recent results.",
    func=search.run,
)

tool.run("Obama's first name?")

"1 Child's First Name. (Type or print). BARACK. CERTIFICATE OF LIVE BIRTH lb ... OBAMA, II. Day. 4. 6b. Island. Year. 5b. Hour. 1961 7:24 P.M.. Oahu. 6d. Is Place\xa0... His last name, Obama, was derived from his Luo descent. Obama's parents met in ... First 100 days. Main article: First 100 days of Barack Obama's presidency. Apr 2, 2018 ... What's Barack Obama's nickname? · His full name is Barack Hussein Obama II. · I have only heard him called by these names: · Barack, Mr. Obama,\xa0... First Lady Michelle LaVaughn Robinson Obama is a lawyer, writer, and the wife of the 44th President, Barack Obama. She is the first African-American First\xa0... Dec 19, 2018 ... His full name is Barack Hussein Obama II. Since the “II” is simply because he was named for his father, his last name is Obama. Barack Hussein Obama II was born August 4, 1961, in Honolulu, Hawaii, to parents Barack H. Obama, Sr., and Stanley Ann Dunham. His parents divorced when he\xa0... Jan 19, 2017 ... Hopeful parents na

# Steps for providing a solution
1. Ask about the stock and provide context
2. Search financial information, news and stock price
3. Provide a summary of the stock
4. Based on the information, provide a recommendation

In [9]:
from langchain.chains import SequentialChain
from langchain.agents import tool, AgentExecutor

llm = OpenAI(api_key=SECRET_KEY, temperature=.7)

@tool
def news_search(stock: str) -> str:
    """Search Google for recent results about the stock's news."""
    return search.run(f"Latest news about {stock}")

@tool
def financial_search(stock: str) -> str:
    """Search Google for recent results about the stock's financials."""
    return search.run(f"Latest financials of {stock}")

financial_search.invoke("Apple Inc. (AAPL)"), news_search.invoke("Apple Inc. (AAPL)")


('Newsroom · Apple Vision Pro arrives in Apple Store locations across the U.S. · Apple reports first quarter results · Apple announces more than 600 new apps built\xa0... Get the detailed quarterly/annual income statement for Apple Inc. (AAPL). Find out the revenue, expenses and profit or loss over the last fiscal year. Apple Inc. balance sheet, income statement, cash flow, earnings & estimates, ratio and margins. View AAPL financial statements in full. Find the latest Apple Inc. (AAPL) stock quote, history, news and other vital information to help you with your stock trading and investing. Find the latest Apple Inc. Common Stock (AAPL) annual income statements, balance sheets, financial statements, ratios, and cash flow data at Nasdaq.com. Aug 3, 2023 ... The Company posted quarterly revenue of $81.8 billion, down 1 percent year over year, and quarterly earnings per diluted share of $1.26, up 5\xa0... 4 days ago ... Apple Inc AAPL ; Valuation · Price/Earnings (Normalized). 29.23, 24.3